# Keras hyperparameter optimization with Talos example

## Useful links for Talos

Official Github [link](https://github.com/autonomio/talos)

Official documents [link](https://autonomio.github.io/docs_talos/#introduction)

Official example can be found here [example link](https://nbviewer.jupyter.org/github/autonomio/talos/blob/master/examples/Hyperparameter%20Optimization%20with%20Keras%20for%20the%20Iris%20Prediction.ipynb#three)






## Import modules

In [0]:
!pip install talos

In [0]:
import numpy as np
import keras
from keras import backend as K
import talos as ta
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, BatchNormalization, MaxPooling2D, Flatten, Activation

## Import data

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 2s 0us/step


## Preprocess data

In [0]:
img_rows, img_cols = 28, 28
num_classes = 10

In [4]:
x_train.shape, y_train.shape, x_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28))

In [5]:
K.image_data_format()

'channels_last'

In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [9]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
x = np.concatenate((x_train, x_test), axis=0)

In [22]:
x.shape

(70000, 28, 28, 1)

In [29]:
 y = np.concatenate((y_train, y_test), axis=0)
y.shape

(70000, 10)

## Prepare model for Talos experiments

### Configuring the Keras Model for Talos
In order to prepare a Keras model for a Talos experiment, we need to do four things:

1.   Add input parameters to the function
2.  Replace the hyperparameter inputs with references to params dictionary
3. Make sure model.fit() stores the history object
4. Modify the output of the model


In [0]:
def fashion_mnist_fn(x_train, y_train, x_val, y_val, params):
    
    # Extract the hyper-params I want to tune from params
    conv_dropout = float(params['conv_dropout'])
    dense1_neuron = int(params['dense1_neuron'])
    activation_tune = params['activation']
    model = Sequential()
    model.add(BatchNormalization(input_shape=x_train.shape[1:]))
    model.add(Conv2D(64, (5, 5), padding='same', activation=activation_tune))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(conv_dropout))

    model.add(BatchNormalization(input_shape=x_train.shape[1:]))
    model.add(Conv2D(128, (5, 5), padding='same', activation=activation_tune))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(conv_dropout))

    model.add(BatchNormalization(input_shape=x_train.shape[1:]))
    model.add(Conv2D(256, (5, 5), padding='same', activation=activation_tune))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(Dropout(conv_dropout))

    model.add(Flatten())
    model.add(Dense(dense1_neuron))
    model.add(Activation(activation_tune))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('softmax'))
  
    model.compile(
        optimizer='adam', # tf.train.AdamOptimizer(learning_rate=1e-3, )
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )


    out = model.fit(
        x, y, epochs=10, batch_size=32, 
        verbose=0,
        validation_data=[x_val, y_val]
    )
    
    #need to return two items. 
    #the first one is history object from model.fit()
    #the second one is model
    return out, model


## Run Talos experiments

In [0]:
# Set params you want to try
para = {
    'dense1_neuron': [256, 512],
    'activation': ['relu', 'elu'],
    'conv_dropout': [0.25, 0.4]
}

While many configurations are possible, the only things that you absolutely must input to a Talos experiment are:



*   x: x = np.concatenate((x_train, x_test), axis=0)
*   y: y = np.concatenate((y_train, y_test), axis=0)
*   params we defined above
*  model function we defined above





In [30]:
scan_results = ta.Scan(x, y, para, fashion_mnist_fn)

  0%|          | 0/8 [00:00<?, ?it/s]WARNING: Logging before flag parsing goes to stderr.
W0811 22:22:16.964628 140386566686592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 22:22:17.003978 140386566686592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 22:22:17.022866 140386566686592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0811 22:22:17.023948 140386566686592 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto 

In [31]:
scan_results.data.head()

,round_epochs,val_loss,val_acc,loss,acc,dense1_neuron,activation,conv_dropout
0,10,0.207849,0.922762,0.289434,0.897057,256,relu,0.40
1,10,0.159383,0.938952,0.258707,0.910071,512,elu,0.25
2,10,0.171147,0.936857,0.230664,0.916829,256,relu,0.25
3,10,0.187549,0.928381,0.291581,0.894986,256,elu,0.40
4,10,0.180831,0.932714,0.289033,0.898457,512,elu,0.40
